# RAG System Workshop: Retrieval & Generation Pipeline

Welcome to Part 2 of the RAG workshop! Now that you've ingested your data, let's build the retrieval and generation components.

## What you'll learn:
1. **Query Processing**: Prepare user queries for retrieval
2. **Semantic Search**: Retrieve relevant chunks from Pinecone
3. **Context Building**: Organize retrieved information
4. **Response Generation**: Use LLMs to generate answers
5. **RAG Orchestration**: Combine everything into a complete system

## Part 1: Setup and Configuration

First, let's set up our environment and load the necessary libraries.

In [ ]:
# Install required packages if not already installed
!pip install pinecone 
!pip install openai
!pip install python-dotenv

In [1]:
import os
import json
from typing import List, Dict, Any, Tuple
from dataclasses import dataclass
import time

# External libraries
from pinecone import Pinecone
import openai
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [30]:
# ============================================
# Configuration (same as ingestion pipeline)
# ============================================

# API Keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # TODO: Ensure your .env file has this
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")  # TODO: Ensure your .env file has this

# Model Configuration
EMBEDDING_MODEL = "text-embedding-3-small"  # For query embedding
GENERATION_MODEL = "gpt-4o-mini"  # TODO: Choose your generation model (gpt-4, gpt-3.5-turbo, etc.)

# Retrieval Configuration
TOP_K_RESULTS = 5  # TODO: Adjust number of chunks to retrieve
SIMILARITY_THRESHOLD = 0.6  # TODO: Minimum similarity score for relevance

# Generation Configuration
MAX_CONTEXT_LENGTH = 3000  # TODO: Maximum characters of context to include
TEMPERATURE = 0.0  # TODO: Adjust creativity (0=deterministic, 1=creative)
MAX_TOKENS = 500  # TODO: Maximum length of generated response

# Pinecone Configuration
PINECONE_INDEX_NAME = "earnings-calls"  # Should match your ingestion pipeline

# Initialize clients
openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

print("✅ Configuration complete!")
print(f"📊 Using index: {PINECONE_INDEX_NAME}")
print(f"🤖 Generation model: {GENERATION_MODEL}")

✅ Configuration complete!
📊 Using index: earnings-calls
🤖 Generation model: gpt-4o-mini


## Part 2: Query Processing

### Step 1: Query Enhancement
Before searching, we can enhance queries to improve retrieval quality.

In [ ]:
def enhance_query(query: str, add_context: bool = True) -> str:
    """
    Enhance a user query for better retrieval.
    
    Args:
        query: Original user query
        add_context: Whether to add contextual keywords
    
    Returns:
        Enhanced query string
    """
    # TODO: Implement query enhancement strategies
    # Ideas:
    # - Add domain-specific keywords
    # - Expand acronyms
    # - Add synonyms
    # - Use a small LLM to rephrase the query
    
    enhanced = query
    
    if add_context:
        # Add earnings call context if not present
        keywords = ['earnings', 'revenue', 'financial', 'quarter', 'fiscal']
        if not any(keyword in query.lower() for keyword in keywords):
            enhanced = f"{query} earnings financial results"
    
    return enhanced

# Test query enhancement
test_query = "iPhone sales"
enhanced = enhance_query(test_query)
print(f"Original: {test_query}")
print(f"Enhanced: {enhanced}")

Original: iPhone sales
Enhanced: iPhone sales earnings financial results


In [6]:
def generate_query_embedding(query: str) -> List[float]:
    """
    Generate embedding for a query using OpenAI.
    
    Args:
        query: Query text
    
    Returns:
        Query embedding vector
    """
    # Clean the query
    query = query.replace("\n", " ").strip()
    
    # Generate embedding
    response = openai_client.embeddings.create(
        input=query,
        model=EMBEDDING_MODEL
    )
    
    return response.data[0].embedding

# Test embedding generation
test_embedding = generate_query_embedding("What is Apple's revenue?")
print(f"✅ Generated embedding with {len(test_embedding)} dimensions")

✅ Generated embedding with 1536 dimensions


## Part 3: Retrieval Pipeline

### Step 2: Semantic Search

In [7]:
@dataclass
class RetrievedChunk:
    """Data class for retrieved chunks"""
    text: str
    score: float
    metadata: Dict[str, Any]
    
    def __str__(self):
        return f"[Score: {self.score:.3f}] {self.metadata.get('ticker', 'N/A')} - {self.metadata.get('date', 'N/A')}"

In [8]:
def retrieve_relevant_chunks(query: str, 
                           top_k: int = TOP_K_RESULTS,
                           filter_dict: Dict = None) -> List[RetrievedChunk]:
    """
    Retrieve relevant chunks from Pinecone.
    
    Args:
        query: User query
        top_k: Number of results to retrieve
        filter_dict: Optional metadata filters (e.g., {'ticker': 'AAPL'})
    
    Returns:
        List of retrieved chunks with scores and metadata
    """
    # Generate query embedding
    query_embedding = generate_query_embedding(query)
    
    # Query Pinecone
    # TODO: Add metadata filtering if needed
    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True,
        filter=filter_dict  # Optional filtering
    )
    
    # Process results into RetrievedChunk objects
    chunks = []
    for match in results['matches']:
        chunk = RetrievedChunk(
            text=match['metadata'].get('text', ''),
            score=match['score'],
            metadata=match['metadata']
        )
        chunks.append(chunk)
    
    return chunks

# Test retrieval
test_query = "What is Apple's revenue growth?"
chunks = retrieve_relevant_chunks(test_query, top_k=3)

print(f"\n🔍 Query: {test_query}")
print(f"📊 Retrieved {len(chunks)} chunks:\n")
for i, chunk in enumerate(chunks, 1):
    print(f"{i}. {chunk}")
    print(f"   Preview: {chunk.text[:100]}...\n")


🔍 Query: What is Apple's revenue growth?
📊 Retrieved 3 chunks:

1. [Score: 0.619] AAPL - 2018-May-01
   Preview:   2. Earnings.
          2. Revenues:
               1. 2Q18, $61.1b.
                    1. Up 16% ...

2. [Score: 0.600] AAPL - 2018-May-01
   Preview:  and Japan, revenue up more than 20%.
               4. iPhone's performance capped tremendous fisca...

3. [Score: 0.598] AAPL - 2018-May-01
   Preview:           8. Had all-time record revenue from App Store, Apple Music, iCloud, Apple Pay and more.
  ...



In [11]:
def filter_chunks_by_relevance(chunks: List[RetrievedChunk], 
                              threshold: float = SIMILARITY_THRESHOLD) -> List[RetrievedChunk]:
    """
    Filter chunks based on similarity score threshold.
    
    Args:
        chunks: List of retrieved chunks
        threshold: Minimum similarity score
    
    Returns:
        Filtered list of relevant chunks
    """
    # TODO: Implement relevance filtering
    # Consider:
    # - Similarity score threshold
    # - Date recency
    # - Source diversity, might not be relevant in this particular case :)
    
    relevant_chunks = [chunk for chunk in chunks if chunk.score >= threshold]
    
    print(f"✅ Filtered {len(chunks)} chunks to {len(relevant_chunks)} relevant chunks")
    print(f"   (Using threshold: {threshold})")
    
    return relevant_chunks

# Test filtering
filtered_chunks = filter_chunks_by_relevance(chunks, threshold=0.6)
print(f"\nRelevant chunks: {len(filtered_chunks)}")

✅ Filtered 3 chunks to 2 relevant chunks
   (Using threshold: 0.6)

Relevant chunks: 2


## Part 4: Context Building

### Step 3: Organize Retrieved Information

In [12]:
def build_context(chunks: List[RetrievedChunk], 
                 max_length: int = MAX_CONTEXT_LENGTH) -> str:
    """
    Build context string from retrieved chunks.
    
    Args:
        chunks: List of retrieved chunks
        max_length: Maximum context length in characters
    
    Returns:
        Formatted context string
    """
    # TODO: Implement smart context building
    # Consider:
    # - Chunk ordering (by score, date, etc.)
    # - Deduplication
    # - Length constraints
    
    context_parts = []
    current_length = 0
    
    for chunk in chunks:
        # Format each chunk with metadata
        chunk_text = f"[{chunk.metadata.get('ticker', 'N/A')} - {chunk.metadata.get('date', 'N/A')}]\n{chunk.text}\n"
        
        # Check if adding this chunk exceeds max length
        if current_length + len(chunk_text) > max_length:
            # Truncate if necessary
            remaining = max_length - current_length
            if remaining > 100:  # Only add if we have reasonable space
                chunk_text = chunk_text[:remaining] + "..."
                context_parts.append(chunk_text)
            break
        
        context_parts.append(chunk_text)
        current_length += len(chunk_text)
    
    context = "\n---\n".join(context_parts)
    
    print(f"📝 Built context with {len(context_parts)} chunks")
    print(f"   Total length: {len(context)} characters")
    
    return context

# Test context building
if filtered_chunks:
    context = build_context(filtered_chunks)
    print(f"\nContext preview:\n{context[:500]}...")

📝 Built context with 2 chunks
   Total length: 1049 characters

Context preview:
[AAPL - 2018-May-01]
  2. Earnings.
          2. Revenues:
               1. 2Q18, $61.1b.
                    1. Up 16% YoverY.
                    2. Sixth consecutive qtr. of accelerating revenue growth.
               2. Broad-based performance with:
                    1. iPhone up 14%.
                    2. Services up 31%.
                    3. Wearables up almost 50%.
               3. Grew in each geographic segment.
                    1. In Greater China and Japan, revenue up more t...


In [18]:
def format_source_citations(chunks: List[RetrievedChunk]) -> str:
    """
    Format source citations from retrieved chunks.
    
    Args:
        chunks: List of retrieved chunks
    
    Returns:
        Formatted citations string
    """
    citations = []
    seen = set()
    
    for chunk in chunks:
        # Create unique citation key
        print(chunk.metadata)
        ticker = chunk.metadata.get('ticker', 'Unknown')
        date = chunk.metadata.get('date', 'Unknown')
        citation_key = f"{ticker}_{date}"
        
        # Avoid duplicates
        #TODO: 
        #1. Can go more granular if needed (e.g., section)
        #2. Citation can also be a webpage url if available and saved as metadata
        if citation_key not in seen:
            seen.add(citation_key)
            citations.append(f"- {ticker} Earnings Call ({date})")
    
    return "\n".join(citations)

# Test citation formatting
if filtered_chunks:
    citations = format_source_citations(filtered_chunks)
    print("📚 Sources:")
    print(citations)

{'chunk_index': 6.0, 'date': '2018-May-01', 'filename': '2018-May-01-AAPL.txt', 'text': '  2. Earnings.\n          2. Revenues:\n               1. 2Q18, $61.1b.\n                    1. Up 16% YoverY.\n                    2. Sixth consecutive qtr. of accelerating revenue growth.\n               2. Broad-based performance with:\n                    1. iPhone up 14%.\n                    2. Services up 31%.\n                    3. Wearables up almost 50%.\n               3. Grew in each geographic segment.\n                    1. In Greater China and Japan, revenue up more than 20%.\n            ', 'ticker': 'AAPL'}
{'chunk_index': 7.0, 'date': '2018-May-01', 'filename': '2018-May-01-AAPL.txt', 'text': " and Japan, revenue up more than 20%.\n               4. iPhone's performance capped tremendous fiscal 1H, with $100b in iPhone revenue.\n                    1. Up $12b over last year, setting new 1H record.\n                    2. Highest 1H growth rate in three years.\n               5. 

## Part 5: Generation Pipeline

### Step 4: Response Generation with LLM

In [19]:
def create_prompt(query: str, context: str) -> str:
    """
    Create a prompt for the LLM with query and context.
    
    Args:
        query: User's question
        context: Retrieved context
    
    Returns:
        Formatted prompt string
    """
    # TODO: Customize this prompt template for your use case
    prompt = f"""You are a financial analyst assistant. Answer the question based on the provided earnings call transcript excerpts.

Context from earnings calls:
{context}

Question: {query}

Instructions:
- Base your answer only on the provided context
- If the context doesn't contain enough information, say so
- Be specific and cite the company and date when referencing information
- Keep the answer concise but comprehensive

Answer:"""
    
    return prompt

# Test prompt creation
if filtered_chunks:
    test_prompt = create_prompt("What is the revenue growth for Apple in the year 2018?", context[:500])
    print("📝 Sample prompt:")
    print(test_prompt[:400] + "...")

📝 Sample prompt:
You are a financial analyst assistant. Answer the question based on the provided earnings call transcript excerpts.

Context from earnings calls:
[AAPL - 2018-May-01]
  2. Earnings.
          2. Revenues:
               1. 2Q18, $61.1b.
                    1. Up 16% YoverY.
                    2. Sixth consecutive qtr. of accelerating revenue growth.
               2. Broad-based performance with:...


In [25]:
def generate_response(query: str, 
                     context: str,
                     model: str = GENERATION_MODEL,
                     temperature: float = TEMPERATURE,
                     max_tokens: int = MAX_TOKENS) -> str:
    """
    Generate a response using OpenAI's GPT model.
    
    Args:
        query: User's question
        context: Retrieved context
        model: OpenAI model to use
        temperature: Creativity parameter
        max_tokens: Maximum response length
    
    Returns:
        Generated response
    """
    # Create the prompt
    prompt = create_prompt(query, context)
    
    # TODO: Add error handling for API calls
    try:
        # Call OpenAI API
        response = openai_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful financial analyst assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        # Extract the response
        answer = response.choices[0].message.content
        
        return answer
    
    except Exception as e:
        print(f"❌ Error generating response: {e}")
        return "I encountered an error while generating the response. Please try again."

# Test response generation
if filtered_chunks:
    test_response = generate_response(
        "What is the revenue growth?",
        context,
        temperature=0.5  # Lower temperature for more focused answers
    )
    print("🤖 Generated Response:")
    print(test_response)

🤖 Generated Response:
The revenue growth for Apple Inc. (AAPL) in the second quarter of 2018 (2Q18) was 16% year-over-year, with total revenues reaching $61.1 billion. This information is from the earnings call on May 1, 2018.


## Part 6: Complete RAG Pipeline

### Step 5: Orchestrate Everything Together

In [26]:
@dataclass
class RAGResponse:
    """Complete RAG response with all components"""
    query: str
    answer: str
    sources: str
    chunks_retrieved: int
    chunks_used: int
    
    def __str__(self):
        return f"""\n{'='*50}
📝 Query: {self.query}
{'='*50}

💡 Answer:
{self.answer}

📚 Sources:
{self.sources}

📊 Statistics:
- Chunks retrieved: {self.chunks_retrieved}
- Chunks used: {self.chunks_used}
{'='*50}"""

In [31]:
def rag_pipeline(query: str,
                top_k: int = TOP_K_RESULTS,
                threshold: float = SIMILARITY_THRESHOLD,
                filter_dict: Dict = None,
                enhance: bool = True) -> RAGResponse:
    """
    Complete RAG pipeline: retrieve, build context, and generate response.
    
    Args:
        query: User's question
        top_k: Number of chunks to retrieve
        threshold: Similarity threshold for filtering
        filter_dict: Optional metadata filters
        enhance: Whether to enhance the query
    
    Returns:
        RAGResponse object with answer and metadata
    """
    print(f"\n🚀 Starting RAG Pipeline")
    print(f"   Query: {query}")
    
    # Step 1: Enhance query (optional)
    if enhance:
        query_enhanced = enhance_query(query)
        print(f"   Enhanced: {query_enhanced}")
    else:
        query_enhanced = query
    
    # Step 2: Retrieve relevant chunks
    print(f"\n📥 Retrieving chunks...")
    chunks = retrieve_relevant_chunks(query_enhanced, top_k=top_k, filter_dict=filter_dict)
    chunks_retrieved = len(chunks)
    
    # Step 3: Filter by relevance
    print(f"\n🔍 Filtering chunks...")
    relevant_chunks = filter_chunks_by_relevance(chunks, threshold=threshold)
    chunks_used = len(relevant_chunks)
    
    if not relevant_chunks:
        return RAGResponse(
            query=query,
            answer="I couldn't find relevant information to answer your question.",
            sources="No relevant sources found.",
            chunks_retrieved=chunks_retrieved,
            chunks_used=0
        )
    
    # Step 4: Build context
    print(f"\n📝 Building context...")
    context = build_context(relevant_chunks)
    
    # Step 5: Generate response
    print(f"\n🤖 Generating response...")
    answer = generate_response(query, context)
    
    # Step 6: Format citations
    sources = format_source_citations(relevant_chunks)
    
    # Create response object
    response = RAGResponse(
        query=query,
        answer=answer,
        sources=sources,
        chunks_retrieved=chunks_retrieved,
        chunks_used=chunks_used
    )
    
    return response

# Test the complete pipeline
test_query = "What is Apple's revenue growth in 2018?"
response = rag_pipeline(test_query)
print(response)


🚀 Starting RAG Pipeline
   Query: What is Apple's revenue growth in 2018?
   Enhanced: What is Apple's revenue growth in 2018?

📥 Retrieving chunks...

🔍 Filtering chunks...
✅ Filtered 5 chunks to 2 relevant chunks
   (Using threshold: 0.6)

📝 Building context...
📝 Built context with 2 chunks
   Total length: 1049 characters

🤖 Generating response...
{'chunk_index': 6.0, 'date': '2018-May-01', 'filename': '2018-May-01-AAPL.txt', 'text': '  2. Earnings.\n          2. Revenues:\n               1. 2Q18, $61.1b.\n                    1. Up 16% YoverY.\n                    2. Sixth consecutive qtr. of accelerating revenue growth.\n               2. Broad-based performance with:\n                    1. iPhone up 14%.\n                    2. Services up 31%.\n                    3. Wearables up almost 50%.\n               3. Grew in each geographic segment.\n                    1. In Greater China and Japan, revenue up more than 20%.\n            ', 'ticker': 'AAPL'}
{'chunk_index': 7.0, 'dat

## 📚 Next Steps

After completing the retrieval and generation pipeline:
1. **Optimize Performance**: Cache embeddings, batch processing
2. **Add Advanced Features**: Multi-turn conversations, feedback loops
3. **Deploy**: Create an API or web interface
4. **Monitor**: Track usage, performance, and accuracy

## 🔑 Key Takeaways

- **Query Processing**: Enhancement improves retrieval quality
- **Semantic Search**: Vector similarity finds relevant content
- **Context Building**: Smart organization improves generation
- **Prompt Engineering**: Good prompts lead to better answers
- **Pipeline Design**: Modular components allow easy improvements
